<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

# 실습 내용

- K-Fold Cross Validation을 사용해 모델의 성능을 예측합니다.

# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [66]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [67]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/Jangrae/csv/master/titanic.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

In [68]:
# 데이터 살펴보기
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [69]:
# 기초통계정보 확인
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) Title 변수 추가**

In [70]:
# 호칭 추출
data['Title'] = data['Name'].str.extract('([A-Za-z]+)\.', expand=False)

In [71]:
data['Title'].value_counts()

Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Major         2
Mlle          2
Col           2
Mme           1
Capt          1
Countess      1
Ms            1
Lady          1
Sir           1
Jonkheer      1
Don           1
Name: Title, dtype: int64

In [72]:
# 대표 호칭 이외는 Others로 변경
main_tit_list = ['Mr', 'Miss', 'Mrs']
data.loc[data['Title'].isin(main_tit_list)==False, ['Title']] = 'Others'

In [73]:
data['Title'].value_counts()

Mr        517
Miss      182
Mrs       125
Others     67
Name: Title, dtype: int64

**2) 결측치 처리**

In [74]:
# Age 결측치를 Title 별 중앙값으로 채우기
data['Age'].fillna(data.groupby('Title')['Age'].transform('median'), inplace=True)

In [75]:
# Embarked 최빈값으로 채우기
data['Embarked'].fillna('S', inplace=True)

**3) AgeGroup 변수 추가**

In [76]:
# 변수 추가
data['AgeGroup'] = pd.cut(data['Age'], 
                          bins=[0, 16, 25, 35, 60, 100], 
                          labels=['0_16', '17_25', '26_35', '36_60', '61_100'])

**4) 불필요한 변수 제거**

In [77]:
# 불필요한 열 제거
drop_cols = ['PassengerId', 'Name', 'Age', 'Ticket', 'Cabin']
data.drop(drop_cols, axis=1, inplace=True)

**5) 가변수화**

In [78]:
# 가변수화
dummy_vars = ['Title', 'AgeGroup', 'Pclass', 'Sex', 'Embarked']
data = pd.get_dummies(data, columns=dummy_vars, drop_first=True)

**6) x, y 분리**

In [79]:
# Target 확인
target = 'Survived'

# 데이터 분리
x = data.drop(target, axis=1)
y = data[target]

**7) 정규화**

> 정규화를 하면 값이, 0 과 1 사이  
> p.s target은 건드리지말기     
> x만 정규화 해주면 된다!

In [80]:
# 정규화
cols = ['SibSp', 'Parch', 'Fare']
x[cols] = (x[cols] - x[cols].min()) / (x[cols].max() - x[cols].min())

# 확인
x.head()

,SibSp,Parch,Fare,Title_Mr,Title_Mrs,Title_Others,AgeGroup_17_25,AgeGroup_26_35,AgeGroup_36_60,AgeGroup_61_100,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S
0,0.125,0.0,0.014151,1,0,0,1,0,0,0,0,1,1,0,1
1,0.125,0.0,0.139136,0,1,0,0,0,1,0,0,0,0,0,0
2,0.000,0.0,0.015469,0,0,0,0,1,0,0,0,1,0,0,1
3,0.125,0.0,0.103644,0,1,0,0,1,0,0,0,0,0,0,1
4,0.000,0.0,0.015713,1,0,0,0,1,0,0,0,1,1,0,1


**8) 학습용, 평가용 데이터 분리**

In [81]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 7:3으로 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,
                                                   random_state=2022)

# 4.성능 예측

- K분할 교차 검증 방법으로 모델 성능을 예측합니다.
- cross_val_score(model, x_train, y_train, cv=n) 형태로 사용합니다.
- cv 옵션에 k값(분할 개수, 기본값=5)을 지정합니다.
- cross_val_score 함수는 넘파이 배열 형태의 값을 반환합니다.
- cross_val_score 함수 반환 값의 평균을 해당 모델의 예측 성능으로 볼 수 있습니다.

**1) Decision Tree**

In [82]:
# 불러오기
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

# 선언하기 
model = DecisionTreeClassifier(max_depth=5, random_state=2022)

# 학습, 예측, 평가하기                                     # cv 의미 : 분할 갯수
cv_score = cross_val_score(model, x_train, y_train, cv=10) # cv=5 : 디폴트값

# 확인
print(cv_score)
print('평균:', cv_score.mean())
print('표준편차:', cv_score.std())

[0.85714286 0.79365079 0.76190476 0.77419355 0.87096774 0.85483871
 0.87096774 0.72580645 0.88709677 0.83870968]
평균: 0.8235279057859701
표준편차: 0.052494104278851095


**2) KNN**

In [83]:
# 불러오기
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# 선언하기 
model = KNeighborsClassifier(n_neighbors=5) # n_neighbors=5 : 디폴트 값


# 학습, 예측, 평가하기                                     # cv 의미 : 분할 갯수
cv_score = cross_val_score(model, x_train, y_train, cv=10) # cv=5 : 디폴트값

# 확인
print(cv_score)
print('평균:', cv_score.mean())
print('표준편차', cv_score.std())

[0.80952381 0.79365079 0.79365079 0.83870968 0.87096774 0.87096774
 0.87096774 0.82258065 0.85483871 0.85483871]
평균: 0.8380696364567332
표준편차 0.029682502585093116


In [84]:
# 성능 정보 수집
score = []
my_cv = []
my_neighbors = []

# cross_val_score 에서 cv값은 k-fold cross validation에서 몇 개의 분할을 사용할 것인지를 정해준다.
# 따라서, cv값이 적어도 2는 되어야 분할이 이루어지기 때문에 (=모든 데이터가 평가에 한 번, 학습에 k-1번 사용)
# range에서 2 부터 시작 한다.

for c in range(2, 21) :
    for n in range(1, 11) :
        model = KNeighborsClassifier(n_neighbors=n)
        cv_score = cross_val_score(model, x_train, y_train, cv=c)
        score.append(cv_score.mean())
        my_cv.append(c)
        my_neighbors.append(n)
    
result = pd.DataFrame()
result['cv'] = my_cv
result['my_neighbors'] = my_neighbors
result['score'] = score
result.sort_values(by='score', ascending=False, inplace=True)
result

,cv,my_neighbors,score
134,15,5,0.839683
135,15,6,0.839683
145,16,6,0.839448
84,10,5,0.838070
95,11,6,0.837890
...,...,...,...
20,4,1,0.757744
30,5,1,0.757742
70,9,1,0.756223
40,6,1,0.756192


> cv=15 이고 my_neigbors=5 일때, 모델성능이 가장 좋다.

**3) Logistic Regression**

- LogisticRegression 사용시 발생하는 Warning을 없애려면 충분한 max_iter를 지정합니다.

In [85]:
# 불러오기
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# 선언하기 
model = LogisticRegression(max_iter=500, random_state=2022) # max_iter=500 : 반복횟수

# 학습, 예측, 평가하기                                     # cv 의미 : 분할 갯수
cv_score = cross_val_score(model, x_train, y_train, cv=10) # cv=5 : 디폴트값

# 확인
print(cv_score)
print('평균:', cv_score.mean())
print('표준편차:', cv_score.std())

[0.76190476 0.77777778 0.74603175 0.79032258 0.87096774 0.82258065
 0.83870968 0.70967742 0.82258065 0.82258065]
평균: 0.7963133640552995
표준편차: 0.04586727707780835


>  K-Fold Cross Validation(=K분할교차검증) 으로 한 결과,  
> KNeighborsClassifier 이 성능이 가장 좋게 나왔다. 그 중에서도 성능이 좋은 파라미터를 for문으로 찾아보자! (KNeighborsClassifier밑에 해봄!)

**4) 모델 만들기**

> 가장 성능이 좋았던 KNeighborsClassifier 로 모델을 만들어 보장

In [86]:
# 불러오기
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report

# 선언하기
model = KNeighborsClassifier(n_neighbors=5) # n_neighbors=5 : 디폴트 값

# 학습하기
model.fit(x_train, y_train)

# 예측하기
y_pred = model.predict(x_test)

# 평가하기
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[152  16]
 [ 34  66]]
              precision    recall  f1-score   support

           0       0.82      0.90      0.86       168
           1       0.80      0.66      0.73       100

    accuracy                           0.81       268
   macro avg       0.81      0.78      0.79       268
weighted avg       0.81      0.81      0.81       268

